![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/taller-meca-aprendizaje%20no%20supervisado_banner%201169%20x%20200%20px%20-05.png)

# Semana 5. Sesión Sincrónica.

El  *cuaderno* tiene como objetivo hacer una introducción a los sistemas de recomendación.

**NO** es necesario editar el archivo o hacer una entrega. Los ejemplos contienen celdas con código ejecutable (`en gris`), que podrá modificar libremente. Esta puede ser una buena forma de aprender nuevas funcionalidades del *cuaderno*, o experimentar variaciones en los códigos de ejemplo.



# Sistemas de Recomendación

## ¿Qué son los sistemas de recomendaciones?


Las preferencias de los individuos suelen seguir patrones que los sistemas de recomendación pueden aprovechar, por ejemplo,

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/iron_man.jpg)

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/thor.jpg)



- Los sistemas de recomendación entonces encuentran patrones que son utilizados para predecir qué otros productos podrían gustarnos y generar sugerencias, de forma tal que  los usuarios encuentren contenido atractivo en un gran corpus.

- Estos sistemas son muy exitosos, por ejemplo según un estudio del 2013 de [McKinsey](https://www.mckinsey.com/industries/retail/our-insights/how-retailers-can-keep-up-with-consumers), el 35% de los artículos comprados en Amazon surgen de estos sistemas de recomendación, y por lo tanto vale la pena estudiarlos cuidadosamente.

## Filtrado Colaborativo Basado en Usuarios.

El filtrado colaborativo aprovecha el poder de la colaboración para generar recomendaciones.

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/Colab.png)


### Ejemplo

Para entender un poco mejor cuál es el problema al que nos enfrentamos, supongamos que tenemos una matriz con 5 usuarios y 5 productos, en este caso restaurantes.

El valor de la celda denota el rating que le dió cada usuario al restaurante. Este valor lo denotamos como $r_{ij}$ que será entonces el rating que le dio el usuario $i$ al restaurante $j$.


In [1]:
#cargamos librerias
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
ratings = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/data/Ratings.csv')

ratings.head()

,restaurant_id,user_id,rating
0,1,1,5.0
1,1,2,4.0
2,1,3,3.0
3,1,4,NaN
4,1,5,NaN


In [2]:
# pivotamos la tabla para tener una matriz de usuarios y restaurantes
r_matrix = ratings.pivot_table(values='rating', index='user_id', columns='restaurant_id')
r_matrix.head()

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,NaN,NaN
2,4.0,3.0,4.0,NaN,NaN
3,3.0,2.0,3.0,NaN,NaN
4,NaN,NaN,NaN,4.0,5.0
5,NaN,NaN,NaN,4.0,5.0


In [3]:
# creamos una matriz de usuarios y restaurantes donde remplazamos los valores nulos por 0
r_matrix_dummy = r_matrix.copy().fillna(0)
r_matrix_dummy

restaurant_id,1,2,3,4,5
user_id,,,,,
1,5.0,4.0,5.0,0.0,0.0
2,4.0,3.0,4.0,0.0,0.0
3,3.0,2.0,3.0,0.0,0.0
4,0.0,0.0,0.0,4.0,5.0
5,0.0,0.0,0.0,4.0,5.0


### Filtrado colaborativo sencillo (simple): medias, y medias ponderadas.


#### Medias

- Esta estrategia consiste en calcular el rating promedio que le asignó cada usuario

In [4]:
 puntuac_promedio = ratings.groupby('restaurant_id').mean()
 pd.DataFrame(puntuac_promedio.iloc[:,1])

,rating
restaurant_id,
1,4.0
2,3.0
3,4.0
4,4.0
5,5.0


#### Medias ponderadas

- Podemos hacer algo más sofisticado usando recomendaciones de usuarios similares?


- Podemos hacer una media ponderada


$$
r_{ur}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'r}}{\sum_{u',u'\neq u}sim(u,u')}
$$



- Es decir la predicción del rating del usuario $u$ para el restaurante $r$, $r_{ur}$, es la suma ponderada de los ratings de los otros usuarios ($u'$) a este restaurante,


- Ponderado por cuán similares son los usuarios $u'$ a $u$.


- Como medimos similitud?

    - Existen múltiples medidas de distancia que se utilizan para medir la similitud.

    - La distancia de coseno, que suele ser la más utilizada en los sistemas de recomendación.

    - Matemáticamente

$$
coseno(x,y)=\frac{x.y'}{|x||y|}
$$


Es decir, es el cociente del producto punto, dividido por las normas de los vectores.

<center>
<img src = "figs/dist_cos.png" alt = "coseno" style = "width: 300px;"/>
</center>


- Si el ángulo es 0  de grados, entonces los vectores se solapan, y el coseno es igual a 1
- Si el ángulo es 90 de grados, los vectores forman un angulo recto, y el coseno es igual a 0.
- Si el ángulo es 180 de grados, los vectores estan en sentido opuesto, y  el coseno es igual a -1.

In [5]:
import math

math.cos(math.radians(0))

1.0

##### Creamos una función recomendadora usando esto

In [6]:
# Importamos cosine_similarity
from sklearn.metrics.pairwise import cosine_similarity

#Calculamos la similitud de coseno
cosine_sim = cosine_similarity(r_matrix_dummy, r_matrix_dummy)

# Transformamos la matriz de similitud en un DataFrame
cosine_sim = pd.DataFrame(cosine_sim, index=r_matrix.index, columns=r_matrix.index)

cosine_sim


user_id,1,2,3,4,5
user_id,,,,,
1,1.000000,0.999630,0.997241,0.0,0.0
2,0.999630,1.000000,0.998891,0.0,0.0
3,0.997241,0.998891,1.000000,0.0,0.0
4,0.000000,0.000000,0.000000,1.0,1.0
5,0.000000,0.000000,0.000000,1.0,1.0


In [7]:
?cosine_similarity

In [8]:
# Definimos una función para calcular el promedio ponderado de los ratings
def cf_user_wmean(user_id, restaurant_id):

    # Primero verificamos si el restaurante está en la matriz
    if restaurant_id in r_matrix:

        #Buscamos las medidas de similitud con los otros usuarios
        sim_scores = cosine_sim[user_id]

        # Obtenemos los ratings no faltantes de la matriz bajo evaluación
        m_ratings = r_matrix[restaurant_id]

         # Obtenemos los índices de los restaurantes sin rating
        idx = m_ratings[m_ratings.isnull()].index

        # Nos quedamos con similitudes y ratings completos
        sim_scores = sim_scores.drop(idx)
        m_ratings = m_ratings.dropna()

        # Calculamos la media ponderada
        wmean_rating = np.dot(sim_scores, m_ratings)/ sim_scores.sum()

    else:
        # Si no tenemos ninguna información retornamos 3
        wmean_rating = 3.0

    return wmean_rating

In [9]:
cf_user_wmean(1,1)

np.float64(4.000920501832994)

In [10]:
sim_scores = cosine_sim[1]
sim_scores

,1
user_id,
1,1.000000
2,0.999630
3,0.997241
4,0.000000
5,0.000000


In [11]:
m_ratings =r_matrix[1]
m_ratings

,1
user_id,
1,5.0
2,4.0
3,3.0
4,NaN
5,NaN


In [12]:
idx = m_ratings[m_ratings.isnull()].index
idx

Index([4, 5], dtype='int64', name='user_id')

Falta entonces

$$
r_{um}=\frac{\sum_{u',u'\neq u}sim(u,u').r_{u'm}}{\sum_{u',u'\neq u}sim(u,u')}
$$


In [13]:
sim_scores1 = sim_scores.drop(idx)

m_ratings = m_ratings.dropna()

# Calculamos la media ponderada
wmean_rating = np.dot(sim_scores1, m_ratings)/ sim_scores1.sum()
wmean_rating

np.float64(4.000920501832994)

#### Filtrado colaborativo basado en embeddings



Estudiemos un ejemplo en el que queremos generar recomendaciones de películas a partir de *embeddings*. Supondremos que nuestra plataforma de streaming: **"SlowFlow"** tiene 4 usuarios y 5 películas: [Batman: El caballero de la noche asciende](https://es.wikipedia.org/wiki/The_Dark_Knight_Rises), [Harry Potter y la Piedra Filosofal](https://es.wikipedia.org/wiki/Harry_Potter_y_la_piedra_filosofal), [Shrek](https://es.wikipedia.org/wiki/Shrek), [Las trillizas de Belleville](https://es.wikipedia.org/wiki/Les_Triplettes_de_Belleville) y [Memento](https://es.wikipedia.org/wiki/Memento).

La siguiente matriz que llamaremos $A$, donde $A\in\mathbb{R}^{ m\times n}$, es decir, las filas son los $m$ usuarios y las columnas las $n$ películas, muestra con un $\checkmark$ las películas que cada usuario vió:

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/colab0.png)


#### Embedding en una sola dimensión

Comencemos representando las películas y los usuarios a un *embedding space*  de una sola dimensión. Por ejemplo, a lo largo del segmento $[-1,1]$ de forma tal que películas e individuos similares estén más cerca.


![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/colab1.png)


![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/colab2b.png)


#### Embeddings en dos dimensiones

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/colab4.png)


![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/colab5.png)


Finalmente

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/colab3b.png)


#### Como obtenemos los embeddings?

Una forma de obtener embeddings es mediante la Descomposición en Valores Singulares (SVD):

$$
A = U\Sigma V'
$$

En el contexto tradicional de SVD,
    
- $U$ es una matriz ortogonal cuyas columnas son los vectores singulares de la matriz original $A$, que están relacionados con los **vectores propios (característicos)** de $AA'$. Estos vectores también tienen una estrecha relación con las **componentes principales** en Análisis de Componentes Principales (PCA), ya que representan las direcciones principales de variabilidad en los datos para las filas de $A$.
- Por otro lado, $V$ es otra matriz ortogonal cuyas columnas son los vectores singulares de $A$, pero asociados con las columnas de la matriz original, representando las direcciones principales de variabilidad en las columnas de $A$. En otras palabras, $U$ y $V$ son matrices que describen cómo las filas y las columnas de $A$ se pueden proyectar en espacios de características latentes.

En el **contexto de embeddings**, $U$ y $V$ tienen una interpretación más específica.
    
- $U$ representa la matriz de **embeddings de los usuarios**, es decir, cada fila de $U$ es una representación latente de un usuario en un espacio de características, capturando sus preferencias a lo largo de diferentes dimensiones latentes.
- Similarmente, $V$ es la matriz de **embeddings de los ítems**, donde cada columna de $V$ describe un ítem en ese mismo espacio latente, permitiendo modelar cómo los ítems están relacionados con esas características latentes identificadas.

Este **espacio de características latentes** es comparable a lo que ocurre en el contexto de reconocimiento facial con las **Eigenfaces**. En Eigenfaces, las imágenes de rostros se proyectan en un espacio de características latentes donde cada dimensión captura un patrón característico de los rostros, como la forma general o las características distintivas. De manera similar, en el caso de embeddings obtenidos a partir de SVD, tanto los usuarios como los ítems se proyectan en un espacio de características latentes que captura los patrones más representativos de sus interacciones, como preferencias o similitudes. En ambos casos, el objetivo es reducir la dimensionalidad mientras se preserva la información más relevante.

La matriz $\Sigma$ sigue conteniendo los valores singulares, que actúan como ponderadores, indicando qué tan importantes son estas características latentes para describir las interacciones entre usuarios e ítems.

Para reducir la dimensionalidad y obtener una representación de rango inferior, seleccionamos solamente las primeras $k$ características de estas matrices, que consideramos capturan de manera más efectiva las preferencias y los gustos de los usuarios.

####  Interpretación de Embeddings

##### Ejemplo 1:

In [14]:
import pandas as pd
import numpy as np

# Cargamos y visualizamos  los datos
pelis = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/data/pelis.csv')
pelis = pelis.set_index("Usuarios")
pelis

,Matrix,Alien,Star Wars,Casablanca,Titanic
Usuarios,,,,,
Juan,1,1,1,0,0
Jose,3,3,3,0,0
Lucas,4,4,4,0,0
Ana,5,5,5,0,0
Martina,0,0,0,4,4
Ines,0,0,0,5,5
Daniel,0,0,0,2,2


In [15]:
np.linalg.matrix_rank(pelis.to_numpy())

np.int64(2)

In [16]:
U, S, Vt = np.linalg.svd(pelis, full_matrices=False)

In [17]:
print(U[:,0:2])

[[-0.14002801  0.        ]
 [-0.42008403  0.        ]
 [-0.56011203  0.        ]
 [-0.70014004  0.        ]
 [ 0.         -0.59628479]
 [ 0.         -0.74535599]
 [ 0.         -0.2981424 ]]


In [18]:
print(Vt[0:2,:])

[[-0.57735027 -0.57735027 -0.57735027 -0.         -0.        ]
 [-0.         -0.         -0.         -0.70710678 -0.70710678]]


In [19]:
n=2
k=2
S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal

print(S2)

[[12.36931688  0.        ]
 [ 0.          9.48683298]]


In [20]:
#Reconstrucción
l=2
pd.DataFrame(np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))).round()

,0,1,2,3,4
0,1.0,1.0,1.0,0.0,0.0
1,3.0,3.0,3.0,0.0,0.0
2,4.0,4.0,4.0,0.0,0.0
3,5.0,5.0,5.0,0.0,0.0
4,0.0,0.0,0.0,4.0,4.0
5,0.0,0.0,0.0,5.0,5.0
6,0.0,0.0,0.0,2.0,2.0


##### Ejemplo 2:

In [22]:
# Cargamos y visualizamos  los datos
pelis2 = pd.read_csv('https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/data/pelis2.csv')
pelis2 = pelis2.set_index("Usuarios")
pelis2

,Matrix,Alien,Star Wars,Casablanca,Titanic
Usuarios,,,,,
Juan,1,1,1,0,0
Jose,3,3,3,0,0
Lucas,4,4,4,0,0
Ana,5,5,5,0,0
Martina,0,2,0,4,4
Ines,0,0,0,5,5
Daniel,0,1,0,2,2


In [23]:
np.linalg.matrix_rank(pelis2.to_numpy())

np.int64(3)

In [24]:
U, S, Vt = np.linalg.svd(pelis2,full_matrices=False)
print('U, S, Vt ='), U, S, Vt

U, S, Vt =


(None,
 array([[-1.37599126e-01, -2.36114514e-02, -1.08084718e-02,
          5.60112034e-01, -3.75734575e-01],
        [-4.12797378e-01, -7.08343543e-02, -3.24254153e-02,
          2.06393278e-01,  7.55974371e-01],
        [-5.50396503e-01, -9.44458057e-02, -4.32338870e-02,
         -7.24808963e-01, -1.84603760e-01],
        [-6.87995629e-01, -1.18057257e-01, -5.40423588e-02,
          3.43988797e-01, -2.30754700e-01],
        [-1.52775087e-01,  5.91100963e-01,  6.53650843e-01,
          2.58497890e-16,  2.00000000e-01],
        [-7.22165140e-02,  7.31311857e-01, -6.78209218e-01,
          0.00000000e+00,  0.00000000e+00],
        [-7.63875433e-02,  2.95550482e-01,  3.26825421e-01,
          1.29248945e-16, -4.00000000e-01]]),
 array([1.24810147e+01, 9.50861406e+00, 1.34555971e+00, 3.04642685e-16,
        0.00000000e+00]),
 array([[-5.62258405e-01, -5.92859901e-01, -5.62258405e-01,
         -9.01335372e-02, -9.01335372e-02],
        [-1.26641382e-01,  2.87705846e-02, -1.26641382e-01,
 

In [25]:
n=3
k=3
S2= np.resize(S,[n,1])*np.eye(n,k) #ponemos los valores singulares en una matriz diagonal
l=3
pd.DataFrame(np.dot(U[:,0:l],np.dot(S2[0:l,0:l],Vt[0:l,:]))).round()

,0,1,2,3,4
0,1.0,1.0,1.0,-0.0,-0.0
1,3.0,3.0,3.0,-0.0,-0.0
2,4.0,4.0,4.0,-0.0,-0.0
3,5.0,5.0,5.0,-0.0,-0.0
4,-0.0,2.0,-0.0,4.0,4.0
5,-0.0,0.0,-0.0,5.0,5.0
6,-0.0,1.0,-0.0,2.0,2.0


# Evaluación Sistemas de Recomendación


- La evaluación de los sistemas de recomendación es otro paso importante para evaluar la efectividad del método.

- En la literatura podemos ver dos tipologías principales: evaluación offline y online.


##  Offline
- La evaluación offline es la que estamos acostumbrados en ML supervisado donde usamos un conjunto de entrenamiento y un conjunto de prueba; y aplicamos la métrica de evaluación apropiada.
Por ejemplo si se trata de etiquetas numéricas, como las calificaciones de 5 estrellas, la forma más común de validar un sistema de recomendación se basa en su valor de predicción, es decir, la capacidad de predecir las calificaciones del usuario. Las funciones estándar como el error cuadrático medio (RMSE), la precisión, las curvas ROC, etc...


##  Online    

- La evaluación online es cuando utilizamos un conjunto de herramientas que nos permite observar las interacciones de los usuarios con el sistema.
- La técnica en línea más común se llama prueba A-B y tiene la ventaja de permitir la evaluación del sistema al mismo tiempo que los usuarios aprenden, compran o juegan con el sistema de recomendación.


![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/AB.jpg)

    
- Esto acerca la evaluación al funcionamiento real del sistema y la hace realmente efectiva cuando el propósito del sistema es cambiar o influir en el comportamiento de los usuarios.
- Para evaluar la prueba, estamos interesados en medir cómo cambia el comportamiento del usuario cuando el usuario interactúa con diferentes sistemas de recomendación. Y require estimación de parámetros causales.

![Clase aprendizaje no supervisado](https://raw.githubusercontent.com/MECA4605-Aprendizaje-no-supervisado/S5_clase_sistemas_recomendacion_I/main/figs/BING.png)



    
- En esta situación, las métricas fuera de línea como RMSE no son lo suficientemente buenas. En este caso, estamos particularmente interesados en la evaluación del objetivo global del sistema de recomendación, la retención de usuario.
